# Chronos Bolt on MUSED-FM using run_musedfm Functions

This notebook demonstrates how to run Chronos Bolt on the MUSED-FM benchmark using the functions from `run_musedfm.py`. This simplifies the outputs to make easier 

In [7]:
import os
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from typing import Optional

# Import MUSED-FM components
from musedfm.data import Benchmark
from musedfm.metrics import MAPE, MAE, RMSE, NMAE

# Import functions from the examples package
from examples import (
    run_models_on_benchmark, 
    compare_model_performance, 
    export_hierarchical_results_to_csv,
    export_results_to_csv
)

print("MUSED-FM components imported successfully!")


MUSED-FM components imported successfully!


In [8]:
# Self-contained ChronosForecast class for the notebook (FIXED VERSION)
class ChronosForecast:
    """
    Chronos forecasting model wrapper for MUSED-FM evaluation.
    This class is self-contained within the notebook.
    """
    
    def __init__(self, model_path: str = "amazon/chronos-bolt-base", device: str = "cuda:0", num_samples: int = 20):
        """
        Initialize Chronos forecast model.
        
        Args:
            model_path: Path to Chronos model (HuggingFace model ID or local path)
            device: Device to run the model on
            num_samples: Number of samples for probabilistic forecasting
        """
        self.model_path = model_path
        self.device = device
        self.num_samples = num_samples
        self.pipeline = None
        self._load_model()
    
    def _load_model(self):
        """Load the Chronos model."""
        try:
            from chronos import BaseChronosPipeline, ForecastType
            
            self.pipeline = BaseChronosPipeline.from_pretrained(
                self.model_path,
                device_map=self.device,
            )
            print(f"Loaded Chronos model: {self.model_path}")
        except ImportError:
            raise ImportError("Chronos package not installed. Please install with: pip install chronos-forecasting")
        except Exception as e:
            raise RuntimeError(f"Failed to load Chronos model: {e}")
    
    def forecast(self, history: np.ndarray, covariates: Optional[np.ndarray] = None, forecast_horizon: Optional[int] = None) -> np.ndarray:
        """
        Generate forecast from historical data using Chronos.
        
        Args:
            history: Historical time series data
            covariates: Optional covariate data (ignored for Chronos)
            forecast_horizon: Number of future points to forecast (default: 1)
            
        Returns:
            Forecast values
        """
        if forecast_horizon is None:
            forecast_horizon = 1
        
        # Convert history to torch tensor
        if isinstance(history, np.ndarray):
            history_tensor = torch.tensor(history, dtype=torch.float32)
        else:
            history_tensor = torch.tensor(np.array(history), dtype=torch.float32)
        
        # Remove NaN values
        history_clean = history_tensor[~torch.isnan(history_tensor)]
        
        if len(history_clean) == 0:
            # If no valid data, return zeros
            return np.zeros(forecast_horizon)
        
        # Ensure we have enough history for forecasting
        if len(history_clean) < 2:
            # If insufficient data, return the last value repeated
            last_value = float(history_clean[-1]) if len(history_clean) > 0 else 0.0
            return np.full(forecast_horizon, last_value)
        
        # Generate forecast using Chronos
        # FIXED: Use predict_quantiles method with proper parameters
        quantiles, mean = self.pipeline.predict_quantiles(
            context=history_clean,
            prediction_length=forecast_horizon,
            quantile_levels=[0.1, 0.5, 0.9]
        )
        
        # Use the median (0.5 quantile) as our forecast
        # Handle different quantiles array structures
        if isinstance(quantiles, torch.Tensor):
            if quantiles.shape[0] >= 3:  # Has all 3 quantiles
                forecast_np = quantiles[1].detach().cpu().numpy()  # 0.5 quantile
            else:  # Only has 1 quantile
                forecast_np = quantiles[0].detach().cpu().numpy()
        else:
            if len(quantiles) >= 3:  # Has all 3 quantiles
                forecast_np = np.array(quantiles[1])  # 0.5 quantile
            else:  # Only has 1 quantile
                forecast_np = np.array(quantiles[0])
        
        # Handle different output shapes
        if forecast_np.ndim > 1:
            # If we have multiple samples, take the mean
            if forecast_np.shape[0] > 1:
                forecast_np = np.mean(forecast_np, axis=0)
            else:
                forecast_np = forecast_np[0]
        
        # Ensure we have the right length
        if len(forecast_np) != forecast_horizon:
            if len(forecast_np) > forecast_horizon:
                forecast_np = forecast_np[:forecast_horizon]
            else:
                # Pad with the last value if needed
                last_val = forecast_np[-1] if len(forecast_np) > 0 else 0.0
                forecast_np = np.pad(forecast_np, (0, forecast_horizon - len(forecast_np)), 'constant', constant_values=last_val)
        
        return forecast_np
            

print("ChronosForecast class defined successfully!")


ChronosForecast class defined successfully!


In [9]:
# Configuration
BENCHMARK_PATH = "/workspace/data/fm_eval_nested/"  # Adjust this path to your MUSED-FM data
MODEL_PATH = "amazon/chronos-bolt-base"  # Chronos Bolt model
DEVICE = "cuda:0"  # Use "cpu" if you don't have CUDA
NUM_SAMPLES = 20  # Number of samples for probabilistic forecasting
MAX_WINDOWS = 2  # Limit windows per dataset for faster testing
OUTPUT_DIR = "./results/chronos_bolt"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Benchmark path: {BENCHMARK_PATH}")
print(f"Model: {MODEL_PATH}")
print(f"Device: {DEVICE}")
print(f"Output directory: {OUTPUT_DIR}")


Benchmark path: /workspace/data/fm_eval_nested/
Model: amazon/chronos-bolt-base
Device: cuda:0
Output directory: ./results/chronos_bolt


In [10]:
# Initialize Chronos model
try:
    chronos_model = ChronosForecast(
        model_path=MODEL_PATH,
        device=DEVICE,
        num_samples=NUM_SAMPLES
    )
    print("Chronos model initialized successfully!")
except Exception as e:
    print(f"Error initializing Chronos model: {e}")
    print("Make sure you have installed chronos-forecasting and have the required dependencies.")


Loaded Chronos model: amazon/chronos-bolt-base
Chronos model initialized successfully!


In [11]:
# Suppress Chronos prediction length warnings
import warnings
warnings.filterwarnings("ignore", 
                       message="We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it.",
                       category=UserWarning,
                       module="chronos")

# Suppress all Chronos warnings
warnings.filterwarnings("ignore", 
                       module="chronos")


In [ ]:
# Import plotting utilities
import matplotlib.pyplot as plt
from musedfm.plotting import plot_window_forecasts, plot_multiple_windows, plot_baseline_comparison

# Run Chronos evaluation with plot data collection (one window per dataset)
print("Starting Chronos evaluation with plot data collection...")

# Define models dictionary (same format as run_musedfm.py)
models = {
    "chronos": {"model": chronos_model, "univariate": True}
}

# Run evaluation using the efficient run_musedfm function with plot data collection
results = run_models_on_benchmark(
    benchmark_path=BENCHMARK_PATH,
    models=models,
    max_windows=MAX_WINDOWS,
    history_length=512,
    forecast_horizon=128,
    stride=256,
    load_cached_counts=True,
    collect_plot_data=True  # Enable plot data collection
)

print("Evaluation completed successfully!")

# Use plot data directly from results
if '_plot_data' in results:
    plot_data = results['_plot_data']
    print(f"\nUsing {len(plot_data)} windows directly from results for plotting")
    print("Plot data ready for visualization!")
else:
    print("No plot data collected during evaluation.")

Starting Chronos evaluation with plot data collection...
Running Multiple Models on Benchmark


Loading KITTI data from /workspace/data/fm_eval_nested/sequential/KITTI
Found 6114 parquet files
Successfully loaded 6114 valid files
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON files
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON files
Dataset aus_electricity not found in data_hierarchy.json
Loading ECL data from /workspace/data/fm_eval_nested/traditional/ecl
Found 1 parquet files
Successfully loaded 1 valid files
/workspace/data/fm_eval_nested/traditional/open_aq [PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/delhi_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/reykjavik_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/rotterdam_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/winnipeg_combined.parquet')]
Domain ALL_DATASETS not found in file hierarchy
successfully counted 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Compare model performance
compare_model_performance(results)



Model Performance Comparison
Model                MAPE (%)   MAE        RMSE       NMAE       Time (s)  
--------------------------------------------------------------------------------
chronos              1312.54    108611419.8610 129233899.7138 -5.0087    222.65    

Best Performance:
  Lowest MAPE: chronos (1312.54%)
  Lowest MAE:  chronos (108611419.8610)
  Lowest RMSE: chronos (129233899.7138)
  Fastest:     chronos (222.65s)


In [ ]:
# Create individual plots for each window
if plot_data:
    print("Creating individual forecast plots...")
    
    # Create plots directory
    plots_dir = os.path.join(OUTPUT_DIR, "plots")
    os.makedirs(plots_dir, exist_ok=True)
    
    for i, data in enumerate(plot_data[:9]):  # Plot first 9 windows
        window = data['window']
        forecast = data['forecast']
        dataset_name = data['dataset_name']
        category = data['category']
        domain = data['domain']
        window_idx = data['window_idx']
        
        # Create forecasts dictionary for the plotting function
        forecasts = {'Chronos Bolt': forecast}
        
        # Create title
        title = f"{dataset_name} - Window {window_idx}\nCategory: {category}, Domain: {domain}"
        
        # Plot the window
        plot_window_forecasts(
            window=window,
            forecasts=forecasts,
            title=title,
            figsize=(12, 6),
            save_path=os.path.join(plots_dir, f"chronos_forecast_{i+1}.png")
        )
        
        print(f"Saved plot {i+1}: {dataset_name} - Window {window_idx}")
    
    print(f"\nIndividual plots saved to: {plots_dir}")
    
else:
    print("No plot data available.")


In [ ]:
# Export hierarchical results to CSV
export_hierarchical_results_to_csv(results, output_dir=OUTPUT_DIR)



Exporting Hierarchical Results to CSV
✓ Cleaned directory: ./results/chronos_bolt/categories
✓ Cleaned directory: ./results/chronos_bolt/domains
✓ Cleaned directory: ./results/chronos_bolt/datasets
✓ Category results saved: chronos_category_results.csv
✓ Dataset results saved: chronos_dataset_results.csv

Hierarchical CSV files saved to:
  Categories: ./results/chronos_bolt/categories/
  Domains: ./results/chronos_bolt/domains/
  Datasets: ./results/chronos_bolt/datasets/


In [ ]:
# Export detailed CSV results
export_results_to_csv(
    benchmark_path=BENCHMARK_PATH,
    models=models,
    max_windows=MAX_WINDOWS,
    output_dir=OUTPUT_DIR,
    history_length=512,
    forecast_horizon=128,
    stride=256,
    load_cached_counts=True
)



Exporting Results to CSV


Loading KITTI data from /workspace/data/fm_eval_nested/sequential/KITTI
Found 6114 parquet files
Successfully loaded 6114 valid files
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON files
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON files
Dataset aus_electricity not found in data_hierarchy.json
/workspace/data/fm_eval_nested/traditional/open_aq [PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/delhi_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/reykjavik_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/rotterdam_combined.parquet'), PosixPath('/workspace/data/fm_eval_nested/traditional/open_aq/winnipeg_combined.parquet')]
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON files
Domain ALL_DATASETS not found in file hierarchy
successfully counted windows from cached JSON 

TypeError: Window.submit_forecast() got an unexpected keyword argument 'univariate'

In [ ]:
# Check matplotlib backend and ensure plots are displayed
import matplotlib
print(f"Matplotlib backend: {matplotlib.get_backend()}")
print(f"Matplotlib version: {matplotlib.__version__}")

# Set backend to inline for Jupyter notebooks
matplotlib.use('inline')
import matplotlib.pyplot as plt

# Enable interactive mode
plt.ion()

# Test with a simple plot
plt.figure(figsize=(8, 4))
plt.plot([1, 2, 3, 4], [1, 4, 2, 3])
plt.title("Test Plot - If you can see this, plotting is working!")
plt.show()

print("Matplotlib configuration complete!")


In [ ]:
# Check if plot files were created
import os
from pathlib import Path

plots_dir = os.path.join(OUTPUT_DIR, "plots")
print(f"Checking plots directory: {plots_dir}")

if os.path.exists(plots_dir):
    plot_files = list(Path(plots_dir).glob("*.png"))
    print(f"Found {len(plot_files)} plot files:")
    for file in plot_files:
        print(f"  - {file.name} ({file.stat().st_size} bytes)")
    
    if plot_files:
        print(f"\nPlots are being saved successfully!")
        print(f"You can find them in: {plots_dir}")
    else:
        print(f"\nNo plot files found. This might indicate an issue with the plotting functions.")
else:
    print(f"Plots directory does not exist: {plots_dir}")
    print("This suggests the plotting code hasn't been executed yet.")


In [ ]:
# Display plots inline if they exist
from IPython.display import Image, display
import os

plots_dir = os.path.join(OUTPUT_DIR, "plots")

if os.path.exists(plots_dir):
    plot_files = list(Path(plots_dir).glob("*.png"))
    
    if plot_files:
        print("Displaying plots inline:")
        print("=" * 50)
        
        # Display first few plots
        for i, plot_file in enumerate(plot_files[:3]):  # Show first 3 plots
            print(f"\n{plot_file.name}:")
            display(Image(str(plot_file)))
            
        if len(plot_files) > 3:
            print(f"\n... and {len(plot_files) - 3} more plots in {plots_dir}")
    else:
        print("No plot files found to display.")
else:
    print("Plots directory does not exist.")


In [ ]:
# Display results summary
if results and 'chronos' in results:
    chronos_results = results['chronos']
    print("\nChronos Results Summary:")
    print(f"Total windows processed: {chronos_results['windows']}")
    print(f"Total time: {chronos_results['time']:.2f} seconds")
    
    if chronos_results['metrics']:
        metrics = chronos_results['metrics']
        print(f"Average MAPE: {metrics.get('MAPE', 'N/A'):.2f}%")
        print(f"Average MAE: {metrics.get('MAE', 'N/A'):.4f}")
        print(f"Average RMSE: {metrics.get('RMSE', 'N/A'):.4f}")
        print(f"Average NMAE: {metrics.get('NMAE', 'N/A'):.4f}")
    
    print(f"\nResults saved to: {OUTPUT_DIR}")
else:
    print("No results available.")


## Summary

This notebook demonstrates how to run Chronos Bolt on the MUSED-FM benchmark using the efficient functions from `run_musedfm.py`. 

### Key Benefits:

1. **Efficient**: Uses optimized evaluation functions instead of custom loops
2. **Self-contained**: ChronosForecast class defined within the notebook
3. **Comprehensive**: Generates hierarchical CSV results and performance comparisons
4. **Minimal code**: Only ~10 cells vs. hundreds of lines in custom evaluation

### Functions Used:

- `run_models_on_benchmark()`: Main evaluation function
- `compare_model_performance()`: Performance comparison
- `export_hierarchical_results_to_csv()`: Hierarchical CSV export
- `export_results_to_csv()`: Detailed CSV export

This approach leverages the existing, well-tested evaluation framework while keeping the notebook clean and efficient.
